### **Data Mining Using SQL**

<font color="red">File access required:</font> In Colab this notebook requires first uploading files **Shop.csv** and **Movies.csv** using the *Files* feature in the left toolbar. If running the notebook on a local computer, simply ensure these files are in the same workspace as the notebook.

In [1]:
!pip install prettytable==0.7.2
!pip install ipython-sql

  Preparing metadata (setup.py) ... done
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13695 sha256=15a11fd92a47242942562a5f6c2465ebbe516caf16869e5c107762c606fdcbf9
  Stored in directory: /root/.cache/pip/wheels/ca/f9/66/1ebeb8cdff2211eebb6fce02957f9e0a9ae3da4b7e65512d1b
Successfully built prettytable
  Attempting uninstall: prettytable
    Found existing installation: prettytable 3.17.0
    Uninstalling prettytable-3.17.0:
      Successfully uninstalled prettytable-3.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.4 MB/s eta 0:00:00


In [2]:
# Set-up
%load_ext sql
%sql sqlite://
import pandas as pd

In [3]:
# Load tables from CSV files
# Shop
with open('Shop.csv') as f: Shop = pd.read_csv(f, index_col=0)
%sql drop table if exists Shop;
%sql --persist Shop
# Movies
with open('Movies.csv') as f: Movies = pd.read_csv(f, index_col=0)
%sql drop table if exists Movies;
%sql --persist Movies

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted movies'

## Shopping dataset - frequent item-sets

In [4]:
%%sql
select * from Shop

 * sqlite://
Done.


TID,item
1,milk
1,eggs
1,juice
2,milk
2,juice
2,cookies
3,eggs
3,chips
4,milk
4,eggs


### Frequent item-sets of two

#### All pairs of items occurring together

In [5]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item

 * sqlite://
Done.


item1,item2
eggs,milk
eggs,juice
juice,milk
juice,milk
cookies,milk
cookies,juice
chips,eggs
eggs,milk
juice,milk
cookies,milk


#### Now with number of transactions they occur together in

In [6]:
%%sql
select T1.item as item1, T2.item as item2, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item

 * sqlite://
Done.


item1,item2,count()
chips,cookies,1
chips,eggs,1
chips,juice,1
chips,milk,1
cookies,juice,2
cookies,milk,2
eggs,juice,1
eggs,milk,2
juice,milk,3


#### Total number of transactions

In [7]:
%%sql
select count(distinct TID) from Shop

 * sqlite://
Done.


count(distinct TID)
5


#### Put it together for frequent item-sets

In [8]:
support = .3
print(support)

0.3


In [9]:
%%sql
select T1.item as item1, T2.item as item2
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item1,item2
cookies,juice
cookies,milk
eggs,milk
juice,milk


### Frequent item-sets of three

#### All triples of items occurring together

In [10]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item

 * sqlite://
Done.


item1,item2,item3
eggs,juice,milk
cookies,juice,milk
cookies,juice,milk
chips,juice,milk
chips,cookies,milk
chips,cookies,juice


#### Now with number of transactions they occur together in

In [11]:
%%sql
select distinct T1.item as item1, T2.item as item2, T3.item as item3, count()
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


item1,item2,item3,count()
chips,cookies,juice,1
chips,cookies,milk,1
chips,juice,milk,1
cookies,juice,milk,2
eggs,juice,milk,1


#### Put it together for frequent item-sets

In [12]:
support = .3
print(support)

0.3


In [13]:
%%sql
select T1.item as item1, T2.item as item2, T3.item as item3
from Shop T1, Shop T2, Shop T3
where T1.TID = T2.TID and T2.TID = T3.TID
and T1.item < T2.item and T2.item < T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item1,item2,item3
cookies,juice,milk


### <font color = 'green'>**Your Turn - Movies dataset frequent item-sets**</font>

In [14]:
%%sql
select * from Movies limit 15

 * sqlite://
Done.


TID,item
145755,The Fault in Our Stars
145755,Boyhood
145755,Big Hero 6
145755,The Imitation Game
145755,Inside Out
117070,Inside Out
239040,Inside Out
177796,Gone Girl
177796,Magic Mike XXL
126875,The Imitation Game


In [22]:
%%sql
select count(distinct TID) as numUsers,
       count(distinct item) as numMovies
from Movies

 * sqlite://
Done.


numUsers,numMovies
1382,123


#### Mine for frequent item-sets of three and four items in the Movies dataset. Find a single support threshold where the number of frequent item-sets of three items is more than 10 but less than 20, and the number of frequent item-sets of four items is more than 0.

In [23]:
support = 0.03
print(f"Support threshold: {support}")


Support threshold: 0.03


In [24]:
%%sql
SELECT
    m1.item AS item1,
    m2.item AS item2,
    m3.item AS item3,
    COUNT(m1.tid) AS support_count
FROM movies m1
JOIN movies m2 ON m1.tid = m2.tid AND m1.item < m2.item
JOIN movies m3 ON m2.tid = m3.tid AND m2.item < m3.item
GROUP BY m1.item, m2.item, m3.item
HAVING support_count >= (SELECT ROUND(COUNT(DISTINCT tid) * 0.03) FROM movies);

 * sqlite://
Done.


item1,item2,item3,support_count
Big Hero 6,Boyhood,Gone Girl,56
Big Hero 6,Boyhood,The Imitation Game,57
Big Hero 6,Fury,Gone Girl,43
Big Hero 6,Fury,The Imitation Game,44
Big Hero 6,Gone Girl,Inside Out,85
Big Hero 6,Gone Girl,The Imitation Game,119
Big Hero 6,Inside Out,The Imitation Game,102
Boyhood,Fury,Gone Girl,43
Boyhood,Fury,The Imitation Game,43
Boyhood,Gone Girl,Inside Out,54


In [25]:
%%sql
SELECT
    m1.item AS item1,
    m2.item AS item2,
    m3.item AS item3,
    m4.item AS item4,
    COUNT(m1.tid) AS support_count
FROM movies m1
JOIN movies m2 ON m1.tid = m2.tid AND m1.item < m2.item
JOIN movies m3 ON m2.tid = m3.tid AND m2.item < m3.item
JOIN movies m4 ON m3.tid = m4.tid AND m3.item < m4.item
GROUP BY m1.item, m2.item, m3.item, m4.item
HAVING support_count >= (SELECT ROUND(COUNT(DISTINCT tid) * 0.03) FROM movies);

 * sqlite://
Done.


item1,item2,item3,item4,support_count
Big Hero 6,Boyhood,Gone Girl,The Imitation Game,46
Big Hero 6,Gone Girl,Inside Out,The Imitation Game,59
Boyhood,Gone Girl,Inside Out,The Imitation Game,42


## Shopping dataset - association rules

### Association rules with one item on the left-hand side

In [26]:
support = .5
confidence = .5
print(support, confidence)

0.5 0.5


#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules; include number of transactions

In [27]:
%%sql
select item, count() as numtrans
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support

 * sqlite://
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### Do the same query but save the result in a table called "Frequents"

In [28]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
3 rows affected.
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [29]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans as LHS_count, count() as both
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

 * sqlite://
Done.


LHS,RHS,LHS_count,both
eggs,chips,3,1
eggs,juice,3,1
eggs,milk,3,2
juice,chips,3,1
juice,cookies,3,2
juice,eggs,3,1
juice,milk,3,3
milk,chips,4,1
milk,cookies,4,2
milk,eggs,4,2


#### Add check for specified confidence to get final result

In [30]:
%%sql
select T1.item as LHS, T2.item as RHS
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item
having 1.0*count() / F.numtrans > :confidence

 * sqlite://
Done.


LHS,RHS
eggs,milk
juice,cookies
juice,milk
milk,juice


### Association rules with two items on the left-hand side

In [31]:
support = .5
confidence = .5
print(support, confidence)

0.5 0.5


#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [32]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
1 rows affected.
Done.


item1,item2,numtrans
juice,milk,3


#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [33]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


LHS1,LHS2,RHS,numtrans,count()
juice,milk,chips,3,1
juice,milk,cookies,3,2
juice,milk,eggs,3,1


#### Add check for specified confidence to get final result

In [34]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having 1.0*count() / F.numtrans > :confidence

 * sqlite://
Done.


LHS1,LHS2,RHS
juice,milk,cookies


## Shopping dataset - association rules with lift instead of confidence

#### Compute overall frequency of each item -- needed for lift calculation

In [35]:
%%sql
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item

 * sqlite://
Done.


item,1.0*count() / (select count(distinct TID) from Shop)
chips,0.4
cookies,0.4
eggs,0.6
juice,0.6
milk,0.8


#### Do the same query but save the result in a table called "ItemFreq"

In [36]:
%%sql
drop table if exists ItemFreq;
create table ItemFreq(item, freq);
insert into ItemFreq
select item, 1.0*count() / (select count(distinct TID) from Shop)
from Shop
group by item;
select * from ItemFreq

 * sqlite://
Done.
Done.
5 rows affected.
Done.


item,freq
chips,0.4
cookies,0.4
eggs,0.6
juice,0.6
milk,0.8


### Association rules with one item on the left-hand side

In [37]:
support = .5
print(support)

0.5


#### First compute frequent item-sets of one item with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions.

In [38]:
%%sql
drop table if exists Frequent;
create table Frequent(item, numtrans);
insert into Frequent
select item, count()
from Shop
group by item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
3 rows affected.
Done.


item,numtrans
eggs,3
juice,3
milk,4


#### All pairs with frequent left-hand side, number of transactions for left-hand side, and number of transactions for both

In [39]:
%%sql
select T1.item as LHS, T2.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Frequent F
where T1.item = F.item
and T1.TID = T2.TID
and T1.item != T2.item
group by T1.item, T2.item

 * sqlite://
Done.


LHS,RHS,numtrans,count()
eggs,chips,3,1
eggs,juice,3,1
eggs,milk,3,2
juice,chips,3,1
juice,cookies,3,2
juice,eggs,3,1
juice,milk,3,3
milk,chips,4,1
milk,cookies,4,2
milk,eggs,4,2


#### Add check for lift > 1 to get final result

In [40]:
%%sql
select T1.item as LHS, T2.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Frequent F, ItemFreq I
where T1.item = F.item
and T1.TID = T2.TID
and T2.item = I.item
and T1.item != T2.item
group by T1.item, T2.item
having (1.0*count() / F.numtrans) / I.freq > 1

 * sqlite://
Done.


LHS,RHS,lift
juice,cookies,1.6666666666666665
juice,milk,1.25
milk,cookies,1.25
milk,juice,1.25


### Association rules with two items on the left-hand side

In [41]:
support = .5
print(support)

0.5


#### First compute frequent item-sets of two items with specified support, as candidate left-hand sides of assocation rules. Save in table "Frequent" along with number of transactions

In [42]:
%%sql
drop table if exists Frequent;
create table Frequent(item1, item2, numtrans);
insert into Frequent
select T1.item, T2.item, count()
from Shop T1, Shop T2
where T1.TID = T2.TID
and T1.item < T2.item
group by T1.item, T2.item
having 1.0*count() / (select count(distinct TID) from Shop) > :support;
select * from Frequent

 * sqlite://
Done.
Done.
1 rows affected.
Done.


item1,item2,numtrans
juice,milk,3


#### All triples with frequent left-hand side, number of transactions for left-hand side, and number of transactions for all three

In [43]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, F.numtrans, count()
from Shop T1, Shop T2, Shop T3, Frequent F
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item

 * sqlite://
Done.


LHS1,LHS2,RHS,numtrans,count()
juice,milk,chips,3,1
juice,milk,cookies,3,2
juice,milk,eggs,3,1


#### Add check for lift > 1 to get final result

In [44]:
%%sql
select T1.item as LHS1, T2.item as LHS2, T3.item as RHS, (1.0*count() / F.numtrans) / I.freq as lift
from Shop T1, Shop T2, Shop T3, Frequent F, ItemFreq I
where T1.item = F.item1 and T2.item = F.item2
and T1.TID = T2.TID and T2.TID = T3.TID and T3.item = I.item
and T1.item != T2.item and T2.item != T3.item and T1.item != T3.item
group by T1.item, T2.item, T3.item
having (1.0*count() / F.numtrans) / I.freq > 1

 * sqlite://
Done.


LHS1,LHS2,RHS,lift
juice,milk,cookies,1.6666666666666665


### <font color = 'green'>**Your Turn - Movies dataset association rules**</font>

#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and confidence thresholds (need not be the same) so the number of association rules is more than 10 but less than 20.


In [49]:
support = 0.03
confidence = 0.65
print(f"Support: {support}, Confidence: {confidence}")

Support: 0.03, Confidence: 0.65


In [50]:
%%sql
DROP TABLE IF EXISTS Frequent;
CREATE TABLE Frequent(
    item1 TEXT,
    item2 TEXT,
    item3 TEXT,
    numtrans INT
);

INSERT INTO Frequent
SELECT T1.item, T2.item, T3.item, COUNT(*)
FROM Movies T1
JOIN Movies T2 ON T1.TID = T2.TID AND T1.item < T2.item
JOIN Movies T3 ON T2.TID = T3.TID AND T2.item < T3.item
GROUP BY T1.item, T2.item, T3.item
HAVING 1.0 * COUNT(*) / (SELECT COUNT(DISTINCT TID) FROM Movies) >= :support;
SELECT * FROM Frequent;


 * sqlite://
Done.
Done.
14 rows affected.
Done.


item1,item2,item3,numtrans
Big Hero 6,Boyhood,Gone Girl,56
Big Hero 6,Boyhood,The Imitation Game,57
Big Hero 6,Fury,Gone Girl,43
Big Hero 6,Fury,The Imitation Game,44
Big Hero 6,Gone Girl,Inside Out,85
Big Hero 6,Gone Girl,The Imitation Game,119
Big Hero 6,Inside Out,The Imitation Game,102
Boyhood,Fury,Gone Girl,43
Boyhood,Fury,The Imitation Game,43
Boyhood,Gone Girl,Inside Out,54


In [52]:
%%sql
SELECT
    T1.item AS LHS1,
    T2.item AS LHS2,
    T3.item AS LHS3,
    T4.item AS RHS
FROM Movies T1
JOIN Movies T2 ON T1.TID = T2.TID
JOIN Movies T3 ON T2.TID = T3.TID
JOIN Movies T4 ON T3.TID = T4.TID
JOIN Frequent F ON T1.item = F.item1 AND T2.item = F.item2 AND T3.item = F.item3
WHERE T4.item NOT IN (F.item1, F.item2, F.item3)
GROUP BY T1.item, T2.item, T3.item, T4.item
HAVING 1.0 * COUNT(*) / F.numtrans >= :confidence;

 * sqlite://
Done.


LHS1,LHS2,LHS3,RHS
Big Hero 6,Boyhood,Gone Girl,The Imitation Game
Big Hero 6,Boyhood,The Imitation Game,Gone Girl
Big Hero 6,Fury,Gone Girl,Inside Out
Big Hero 6,Fury,Gone Girl,The Imitation Game
Big Hero 6,Fury,The Imitation Game,Gone Girl
Big Hero 6,Gone Girl,Inside Out,The Imitation Game
Boyhood,Fury,Gone Girl,The Imitation Game
Boyhood,Fury,The Imitation Game,Big Hero 6
Boyhood,Fury,The Imitation Game,Gone Girl
Boyhood,Gone Girl,Inside Out,The Imitation Game


#### Mine for association rules in the Movies dataset with three items on the left-hand side. Find support and lift thresholds so the number of association rules is more than 10 but less than 20.


In [55]:
support = 0.015
lift = 3.3
print(f"Support: {support}, Lift: {lift}")

Support: 0.015, Lift: 3.3


In [56]:
%%sql
DROP TABLE IF EXISTS ItemFreq;
CREATE TABLE ItemFreq(
    item TEXT,
    freq REAL
);
INSERT INTO ItemFreq
SELECT
    item,
    1.0 * COUNT(*) / (SELECT COUNT(DISTINCT TID) FROM Movies) AS freq
FROM Movies
GROUP BY item;
SELECT * FROM ItemFreq;

 * sqlite://
Done.
Done.
123 rows affected.
Done.


item,freq
400 Days,0.002170767004341534
A Walk in the Woods,0.000723589001447178
About Alex,0.005065123010130246
Action Jackson,0.000723589001447178
American Ultra,0.013024602026049204
Annie,0.004341534008683068
Anti-Social,0.000723589001447178
Appropriate Behaviour,0.001447178002894356
Ascension,0.002170767004341534
Aziz Ansari: Live at Madison Square Garden,0.000723589001447178


In [57]:
%%sql
DROP TABLE IF EXISTS Frequent;
CREATE TABLE Frequent(
    item1 TEXT,
    item2 TEXT,
    item3 TEXT,
    numtrans INT
);
INSERT INTO Frequent
SELECT
    T1.item AS item1,
    T2.item AS item2,
    T3.item AS item3,
    COUNT(*) AS numtrans
FROM Movies T1
JOIN Movies T2 ON T1.TID = T2.TID AND T1.item < T2.item
JOIN Movies T3 ON T2.TID = T3.TID AND T2.item < T3.item
GROUP BY T1.item, T2.item, T3.item
HAVING 1.0 * COUNT(*) / (SELECT COUNT(DISTINCT TID) FROM Movies) > :support;
SELECT * FROM Frequent;

 * sqlite://
Done.
Done.
39 rows affected.
Done.


item1,item2,item3,numtrans
Big Hero 6,Boyhood,Fury,29
Big Hero 6,Boyhood,Gone Girl,56
Big Hero 6,Boyhood,Inside Out,39
Big Hero 6,Boyhood,The Fault in Our Stars,23
Big Hero 6,Boyhood,The Imitation Game,57
Big Hero 6,Fury,Gone Girl,43
Big Hero 6,Fury,Inside Out,34
Big Hero 6,Fury,The Imitation Game,44
Big Hero 6,Gone Girl,Inside Out,85
Big Hero 6,Gone Girl,The Fault in Our Stars,22


In [58]:
%%sql
SELECT
    T1.item AS LHS1,
    T2.item AS LHS2,
    T3.item AS LHS3,
    T4.item AS RHS,
    (1.0 * COUNT(*) / F.numtrans) / I.freq AS lift_value
FROM Movies T1
JOIN Movies T2 ON T1.TID = T2.TID
JOIN Movies T3 ON T2.TID = T3.TID
JOIN Movies T4 ON T3.TID = T4.TID
JOIN Frequent F ON T1.item = F.item1 AND T2.item = F.item2 AND T3.item = F.item3
JOIN ItemFreq I ON T4.item = I.item
WHERE T4.item NOT IN (F.item1, F.item2, F.item3)
GROUP BY T1.item, T2.item, T3.item, T4.item
HAVING
    1.0 * COUNT(*) / (SELECT COUNT(DISTINCT TID) FROM Movies) >= :support
    AND lift_value > :lift;


 * sqlite://
Done.


LHS1,LHS2,LHS3,RHS,lift_value
Big Hero 6,Boyhood,Gone Girl,Fury,4.828416149068323
Big Hero 6,Boyhood,The Imitation Game,Fury,4.919399949148232
Big Hero 6,Fury,Gone Girl,Boyhood,3.805997552019584
Big Hero 6,Fury,The Imitation Game,Boyhood,3.85725677830941
Big Hero 6,Gone Girl,Inside Out,Fury,3.4167092924126172
Boyhood,Gone Girl,Inside Out,Fury,4.450885668276972
Boyhood,Gone Girl,The Imitation Game,Fury,4.111212814645309
Boyhood,Gone Girl,The Imitation Game,Wild Tales,4.7065015479876156
Boyhood,Inside Out,The Imitation Game,Fury,4.429487179487179
Fury,Gone Girl,Inside Out,Boyhood,3.636842105263158
